In [1]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
data_dir = os.path.join(os.getcwd(), '../boreal_data')

carbon = pd.read_csv(os.path.join(data_dir, 'Carbon_storage.csv'))
HA = pd.read_csv(os.path.join(data_dir, 'Combined_HA.csv'))
deadwood = pd.read_csv(os.path.join(data_dir, 'Deadwood_volume.csv'))
revenue = pd.read_csv(os.path.join(data_dir, 'Timber_revenues.csv'))

In [3]:
carbon_clean = carbon.dropna(axis=0, how='any')
HA_clean = HA.dropna(axis=0, how='any')
deadwood_clean = deadwood.dropna(axis=0, how='any')
revenue_clean = revenue.dropna(axis=0, how='any')

# Optimization

### Carbon without Nans

Lets solve the problem with just one objective and using data without Nan-values

In [4]:
%%time
from Boreal import BorealSolver

bsolver = BorealSolver('glpk', False)
bsolver.solveBoreal(carbon_clean)

CPU times: user 15.3 s, sys: 248 ms, total: 15.6 s
Wall time: 1min 11s


In [5]:
res_dict = dict()
for i in bsolver.model.I:
    for j in bsolver.model.J:
        res_dict[j] = res_dict.get(j,0) + int(bsolver.model.x[i,j].value)
print('Handling, # of stands')        
for key in res_dict:
    print("{:8} {}".format(list(carbon_clean)[key], res_dict[key]))

Handling, # of stands
BAU      19
SA       12775
EXT10    140
EXT30    164
GTR30    67
NTSR     19
NTLR     3609


So it looks like we really are able to solve this problem using the original data! (Without Nans)

### Carbon where Nan:s as Zeros

Lets replace Nan:s with 0:s and lets try to solve the problem. All the single optimization tasks are maximizing, so in the final solution there should be no zeros anyway. (We can check that later)

In [6]:
carbon_zeros = carbon.copy()
carbon_zeros[carbon_zeros.isnull()] = 0

In [7]:
%%time
from Boreal import BorealSolver

zero_bsolver = BorealSolver('glpk', False)
zero_bsolver.solveBoreal(carbon_zeros)

CPU times: user 27.2 s, sys: 284 ms, total: 27.5 s
Wall time: 3min 50s


In [8]:
res_dict = dict()
for i in zero_bsolver.model.I:
    for j in zero_bsolver.model.J:
        res_dict[j] = res_dict.get(j,0) + int(zero_bsolver.model.x[i,j].value)
print('Handling, # of stands')        
for key in res_dict:
    print("{:8} {}".format(list(carbon_clean)[key], res_dict[key]))

Handling, # of stands
BAU      2681
SA       22905
EXT10    174
EXT30    168
GTR30    79
NTSR     48
NTLR     3611


In [9]:
lst = []
for i in zero_bsolver.model.I:
    for j in zero_bsolver.model.J:
        if zero_bsolver.model.x[i,j].value == 1:
            lst.append(carbon.iloc[i,j])

In [10]:
min(lst)

0.70306000000000002

In [11]:
max(lst)

2059.9899999999998

In [12]:
np.min(carbon.dropna(axis=0, how='any').values)

0.52839099999999994

In [13]:
np.max(carbon.dropna(axis=0, how='any').values)

2059.9899999999998

# Clustering

### Feature selection

In [14]:
carbon.corr()

,BAU,SA,EXT10,EXT30,GTR30,NTSR,NTLR
BAU,1.000000,0.893154,0.986115,0.936001,0.996940,0.970110,0.945243
SA,0.893154,1.000000,0.911547,0.984990,0.933510,0.849338,0.852711
EXT10,0.986115,0.911547,1.000000,0.949011,0.984064,0.942080,0.946669
EXT30,0.936001,0.984990,0.949011,1.000000,0.951313,0.873825,0.874188
GTR30,0.996940,0.933510,0.984064,0.951313,1.000000,0.960060,0.934638
NTSR,0.970110,0.849338,0.942080,0.873825,0.960060,1.000000,0.947336
NTLR,0.945243,0.852711,0.946669,0.874188,0.934638,0.947336,1.000000


### Clustering according to the features